In [2]:
from vtk import *

## Load data
#######################################
reader = vtkXMLImageDataReader()
reader.SetFileName('Data/Isabel_2D.vti')
reader.Update()
data = reader.GetOutput()

numCells = data.GetNumberOfCells()
dataArr = data.GetPointData().GetArray('Pressure')
# Enter the contour value in C
C = 100
points = vtkPoints()
cells = vtkCellArray()

for cellId in range(numCells):
    cell = data.GetCell(cellId)
    pid1 = cell.GetPointId(0)
    pid2 = cell.GetPointId(1)
    pid3 = cell.GetPointId(3)
    pid4 = cell.GetPointId(2)
    
    cor1 = data.GetPoint(pid1)
    cor2 = data.GetPoint(pid2)
    cor3 = data.GetPoint(pid3)
    cor4 = data.GetPoint(pid4)
    
    val1 = dataArr.GetTuple1(pid1)
    val2 = dataArr.GetTuple1(pid2)
    val3 = dataArr.GetTuple1(pid3)
    val4 = dataArr.GetTuple1(pid4)
    flag = [0,0,0,0]
    xp = [(),(),(),()]
    if(min(val1,val2)<C<max(val1,val2)):
            xp1 = tuple(x1 - x2 for x1, x2 in zip(cor2, cor1))
            temp = (val1-C)/(val1-val2)
            xp1 = (xp1[0]*temp, xp1[1]*temp, xp1[2]*temp)
            xp1 = (xp1[0]+cor1[0], xp1[1]+cor1[1], xp1[2]+cor1[2])
            flag[0]=1
            xp[0] = xp1
#             print(xp1)
    if(min(val2,val3)<C<max(val2,val3)):
            xp2 = tuple(x1 - x2 for x1, x2 in zip(cor3, cor2))
            temp = (val2-C)/(val2-val3)
            xp2 = (xp2[0]*temp, xp2[1]*temp, xp2[2]*temp)
            xp2 = (xp2[0]+cor2[0], xp2[1]+cor2[1], xp2[2]+cor2[2])
            flag[1]=1
            xp[1] = xp2
#             print(xp2)
    if(min(val3,val4)<C<max(val3,val4)):
            xp3 = tuple(x1 - x2 for x1, x2 in zip(cor4, cor3))
            temp = (val3-C)/(val3-val4)
            xp3 = (xp3[0]*temp, xp3[1]*temp, xp3[2]*temp)
            xp3 = (xp3[0]+cor3[0], xp3[1]+cor3[1], xp3[2]+cor3[2])
            flag[2]=1
            xp[2] = xp3
#             print(xp3)
    if(min(val4,val1)<C<max(val4,val1)):
            xp4 = tuple(x1 - x2 for x1, x2 in zip(cor1, cor4))
            temp = (val4-C)/(val4-val1)
            xp4 = (xp4[0]*temp, xp4[1]*temp, xp4[2]*temp)
            xp4 = (xp4[0]+cor4[0], xp4[1]+cor4[1], xp4[2]+cor4[2])
            flag[3]=1
            xp[3] = xp4
#             print(xp4)
#     print(xp)
    if(sum(flag)==2):
        polyLine = vtkPolyLine()
        polyLine.GetPointIds().SetNumberOfIds(2)
        lineId=0
        for i in range(4):
            if(flag[i]==1):
                polyLine.GetPointIds().SetId(lineId, points.InsertNextPoint(xp[i]))
                lineId = lineId+1
        cells.InsertNextCell(polyLine)
    elif(sum(flag)==4):
        polyLine1 = vtkPolyLine()
        polyLine1.GetPointIds().SetNumberOfIds(2)
        polyLine1.GetPointIds().SetId(0, points.InsertNextPoint(xp[0]))
        polyLine1.GetPointIds().SetId(1, points.InsertNextPoint(xp[1]))
        
        polyLine2 = vtkPolyLine()
        polyLine2.GetPointIds().SetNumberOfIds(2)
        polyLine2.GetPointIds().SetId(0, points.InsertNextPoint(xp[2]))
        polyLine2.GetPointIds().SetId(1, points.InsertNextPoint(xp[3]))
        cells.InsertNextCell(polyLine1)
        cells.InsertNextCell(polyLine2)

        
### Create polydata
####################
pdata = vtkPolyData()

### Add points and cells to polydata
####################################
pdata.SetPoints(points)
pdata.SetLines(cells)

### Store the polydata into a vtkpolydata file with extension .vtp
###################################################################
writer = vtkXMLPolyDataWriter()
writer.SetInputData(pdata)
writer.SetFileName('polyline.vtp')
writer.Write()

1